In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-impact-on-jobs-2030/AI_Impact_on_Jobs_2030.csv


In [5]:
 import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

# --------------------------------------------------
# 1. AUTO CSV FINDER (NO PATH ERROR)
# --------------------------------------------------
csv_path = None

SEARCH_PATHS = ["/kaggle/input", "."]

for base in SEARCH_PATHS:
    for root, dirs, files in os.walk(base):
        for file in files:
            if file.endswith(".csv"):
                csv_path = os.path.join(root, file)
                break
        if csv_path:
            break
    if csv_path:
        break

if csv_path is None:
    print("❌ CSV file not found. Please attach dataset.")
    exit()

print("✅ CSV found at:", csv_path)

# --------------------------------------------------
# 2. LOAD DATASET
# --------------------------------------------------
df = pd.read_csv(csv_path)
print("Dataset shape:", df.shape)

# --------------------------------------------------
# 3. TARGET CREATION (BINARY)
# --------------------------------------------------
df["Target"] = (df["Automation_Probability_2030"] >= 0.5).astype(int)

X = df.drop(columns=["Automation_Probability_2030", "Target"])
y = df["Target"]

# --------------------------------------------------
# 4. PREPROCESSING
# --------------------------------------------------
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

X_processed = preprocessor.fit_transform(X)

# --------------------------------------------------
# 5. TRAIN TEST SPLIT
# --------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42
)

# --------------------------------------------------
# 6. ANN MODEL
# --------------------------------------------------
model = Sequential([
    Dense(32, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(16, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=SGD(learning_rate=0.01),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# --------------------------------------------------
# 7. TRAIN MODEL
# --------------------------------------------------
model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=16,
    validation_data=(X_test, y_test)
)

model.save("ann_jobs_model.h5")
print("✅ ANN model trained and saved successfully")


✅ CSV found at: /kaggle/input/ai-impact-on-jobs-2030/AI_Impact_on_Jobs_2030.csv
Dataset shape: (3000, 18)
Epoch 1/30


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-12-17 12:21:19.001532: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5190 - loss: 0.7117 - val_accuracy: 0.5033 - val_loss: 0.7008
Epoch 2/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5270 - loss: 0.6933 - val_accuracy: 0.5367 - val_loss: 0.6912
Epoch 3/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5660 - loss: 0.6853 - val_accuracy: 0.5533 - val_loss: 0.6845
Epoch 4/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5724 - loss: 0.6805 - val_accuracy: 0.5667 - val_loss: 0.6778
Epoch 5/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5915 - loss: 0.6753 - val_accuracy: 0.6017 - val_loss: 0.6703
Epoch 6/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5990 - loss: 0.6654 - val_accuracy: 0.6300 - val_loss: 0.6594
Epoch 7/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6500 - loss: 0.6521 - val_accuracy: 0.6500 - val_loss: 0.6453
Epoch 8/30
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6661 - loss: 0.6355 - val_accuracy: 0.6650 - val_

✅ ANN model trained and saved successfully
